## Exercice 3 (4 pts)

A partir du fichier "ner_dataset.csv", entraîner un modèle CRF-linéaire pour la détection d'entités nommées. 

Vous diviserez votre dataset de telle sorte à avoir un dataset d'entraînement, de validation et de test. Vous utiliserez les datasets de validation et de test pour contrôler la qualité de votre modèle.

Vous entraînerez plusieurs modèles, en testant différentes features. Vous sauvegarderez le meilleur modèle sur le disque à l'aide de la libraire "joblib". 


In [25]:
import pandas as pd

# Charger le fichier csv
df = pd.read_csv('ner_dataset.csv')

In [26]:
# Suppression des lignes contenant des valeurs manquantes
df.dropna(inplace=True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73362 entries, 0 to 74142
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   token   73362 non-null  object
 1   class   73362 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


In [28]:
# Affichage des dernières lignes du DataFrame
df.tail()

token   class
74138         .       O
74139     Scifo  B-pers
74140         .       O
74141  Dewildec  B-pers
74142         .       O

In [29]:
df['class'].value_counts()

class
O         66340
I-pers     2464
B-loc      1450
B-pers     1118
I-loc       633
I-org       484
B-org       388
I-time      176
B-prod      105
I-prod      105
B-time       99
Name: count, dtype: int64

In [30]:
# Regarder la distribution des classes dans la colonne 'class'

df['class'].value_counts().plot.bar()

<Axes: xlabel='class'>

In [31]:
# Fonction pour transformer le DataFrame en une liste de phrases
def df_to_list_of_sentences(df):
    sentences = []
    current_sentence = []

    for _, row in df.iterrows():
        # Ajouter chaque mot avec son étiquette à la phrase en cours
        current_sentence.append((row['token'], row['class']))

        # Si le mot est une période, considérer que c'est la fin de la phrase
        if row['token'] == '.':
            sentences.append(current_sentence)
            current_sentence = []

    return sentences

# Transformer l'ensemble des données en liste de phrases
all_data_list_of_sentences = df_to_list_of_sentences(df)

In [32]:
print(all_data_list_of_sentences[0])

[('NOUVELLES', 'O'), ('SUISSES', 'O'), ('—', 'O'), ('En', 'B-time'), ('1887', 'I-time'), (',', 'O'), ('la', 'O'), ('Société', 'B-org'), ('suisse', 'I-org'), ('du', 'I-org'), ('Grutli', 'I-org'), ('s', 'O'), ("'", 'O'), ('est', 'O'), ('accrue', 'O'), ('de', 'O'), ('40', 'O'), ('sections', 'O'), (';', 'O'), ('l', 'O'), ("'", 'O'), ('association', 'O'), ('compte', 'O'), ('actuellement', 'O'), ('12,000', 'O'), ('membres', 'O'), ('.', 'O')]


In [33]:
# Diviser les données en ensembles d'entraînement, de validation et de test
from sklearn.model_selection import train_test_split
X_train, rest_data = train_test_split(all_data_list_of_sentences, test_size=0.3)
X_val, X_test = train_test_split(rest_data, test_size=0.5)

In [34]:
X_train

[[('Une', 'O'),
  ('dernière', 'O'),
  ('tentative', 'O'),
  ('de', 'O'),
  ('Cédric', 'B-pers'),
  ('Vasseur', 'I-pers'),
  ('annihilée', 'O'),
  (',', 'O'),
  ('le', 'O'),
  ('sprint', 'O'),
  ('devenait', 'O'),
  ('inéluctable', 'O'),
  ('.', 'O')],
 [('Bâle', 'I-org'),
  (',', 'O'),
  ('H', 'B-org'),
  ('C', 'I-org'),
  ('Bâle', 'I-org'),
  ('Berne', 'B-loc'),
  ('et', 'O'),
  ('Nordstern', 'B-org'),
  ('.', 'O')],
 [('.', 'O')],
 [('Lue', 'O'),
  ('convention', 'O'),
  ('dans', 'O'),
  ('ce', 'O'),
  ('genre', 'O'),
  ('fut', 'O'),
  ('signée', 'O'),
  ('immédiatement', 'O'),
  ('.', 'O')],
 [('Le', 'O'),
  ('Reichstag', 'B-org'),
  ('a', 'O'),
  ('discuté', 'O'),
  (',', 'O'),
  ('en', 'O'),
  ('première', 'O'),
  ('lecture', 'O'),
  (',', 'O'),
  ('deux', 'O'),
  ('propositions', 'O'),
  ('émanées', 'O'),
  (',', 'O'),
  ('l', 'O'),
  ("'", 'O'),
  ('une', 'O'),
  ('du', 'O'),
  ('centre', 'O'),
  (',', 'O'),
  ('l', 'O'),
  ("'", 'O'),
  ('autre', 'O'),
  ('des', 'O'),
  ('cons

In [35]:
from typing import List


def word2features(sent:List[tuple[str]], i:int) -> dict:
    """
    Extrait les caractéristiques du mot actuel dans la phrase, ainsi que, si possible, celles des mots précédents et suivants.
    Étant donné que nous entraînons un modèle CRF linéaire, il n'est pas nécessaire d'extraire des caractéristiques au-delà d'un mot
    avant ou après.

    :param sent: Liste de phrases contenant les tokens
    :type sent: List[Tuple[str]]
    :param i: Index du mot
    :type i: int
    :return: Dictionnaire des caractéristiques du mot
    :rtype: Dict
    """
    word = sent[i][0]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        # features du mot suivant
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isdigit()': word1.isdigit()
        })
    else:
        # BOS : beginning of sentence : le mot est le premier de la phrase
        features['BOS'] = True

    if i < len(sent)-1:
        # features du mot précédent
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.isdigit()': word1.isdigit()
        })
    else:
        # EOS : end of sentence : le mot est le dernier de la phrase
        features['EOS'] = True

    return features




In [36]:
def sent2labels(sent:List[tuple[str]], type:str="ner") -> List[str]:
    """
    Extrait les étiquettes de chaque mot dans la phrase, soit POS, soit NER.

    :param sent: Liste de phrases à traiter
    :type sent: List[Tuple[str]]
    :param type: Type d'étiquette à extraire, soit "pos" soit "ner", par défaut "pos"
    :type type: str, optionnel
    :return: Liste des étiquettes
    :rtype: List[str]
    """

    if type == "pos":
        return [postag for token, postag, label in sent]
    elif type == "ner":
        return [label for token, label in sent]


    
def sent2features(sent:List[List[tuple[str]]]) -> List[dict]:
    """
    Simple fonction pour extraire les features des mots d'une phrase

    :param sent: liste de phrase à traiter
    :type sent: List[List[tuple[str]]]
    :return: liste des features pour chaque mot
    :rtype: List[dict]
    """
    return [word2features(sent, i) for i in range(len(sent))]

def processPipeline(sent:List[List[tuple[str]]], type='ner') -> tuple[List[List[dict]], List[List[str]]]:
    """
    Transforme un jeu de données au format requis, en extrayant les features et les classes

    :param sent: Liste de phrases à traiter
    :type sent: List[List[tuple[str]]]
    :param type: Typle de classe à extraire, soit "pos" soit "ner", defaults to 'pos'
    :type type: str, optional
    :return: Tuple contenant X puis y
    :rtype: tuple[List[List[dict]], List[List[str]]]
    """
    X = [sent2features(x) for x in sent]
    y = [sent2labels(x) for x in sent]

    return X, y

In [37]:
X_train, y_train = processPipeline(X_train, type="ner")
X_val, y_val = processPipeline(X_val, type="ner")
X_test, y_test = processPipeline(X_test, type="ner")

In [38]:
X_train[0]

[{'word.lower()': 'une',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': 'dernière',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': 'dernière',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'une',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:word.isdigit()': False,
  '+1:word.lower()': 'tentative',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': 'tentative',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'dernière',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.isdigit()': False,
  '+1:word.lower()': 'de',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': 'de',
  'word.isupper()': False,

In [39]:
y_train

[['O', 'O', 'O', 'O', 'B-pers', 'I-pers', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['I-org', 'O', 'B-org', 'I-org', 'I-org', 'B-loc', 'O', 'B-org', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-pers', 'O', 'B-pers', 'O'],
 ['O',

In [40]:
import sklearn_crfsuite
# Initialiser et entraîner le modèle CRF
crf = sklearn_crfsuite.CRF()

# Pour éviter l'erreur
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass


In [41]:
from sklearn_crfsuite.metrics import flat_precision_score, flat_recall_score, flat_f1_score
# Évaluer le modèle sur les ensembles de validation et de test
dev_pred = crf.predict(X_val)
print('P :', flat_precision_score(y_val, dev_pred, average='micro'))
print('R :', flat_recall_score(y_val, dev_pred, average='micro'))
print('F1 :', flat_f1_score(y_val, dev_pred, average='micro'))

P : 0.9489296081277213
R : 0.9489296081277213
F1 : 0.9489296081277213


In [42]:
test_pred = crf.predict(X_test)
print('P :', flat_precision_score(y_test, test_pred, average='micro'))
print('R :', flat_recall_score(y_test, test_pred, average='micro'))
print('F1 :', flat_f1_score(y_test, test_pred, average='micro'))

P : 0.9397056048311002
R : 0.9397056048311002
F1 : 0.9397056048311002


In [43]:
# Sauvegarde le modèle

from joblib import dump

dump(crf, 'modele_NER.joblib')

['modele_NER.joblib']